In [72]:
import pandas as pd  
import collections
import numpy as np
from textblob import TextBlob # pip install -U textblob, python -m textblob.download_corpora
from textblob.classifiers import NaiveBayesClassifier


# making dataframe
df = pd.read_csv("data/sentiment140_with_location.csv", names=['target','id','date','flag','user','text', 'location'])  

df

,target,id,date,flag,user,text,location
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",China
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,Dominican Republic
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,Germany
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,Brazil
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",Argentina
...,...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,Japan
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,Spain
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,Nigeria
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,Argentina


In [50]:
# split dataset
shuffled = df.sample(frac = 1)
result = np.array_split(shuffled, 8)
split_df = result[0]
split_df

,target,id,date,flag,user,text,location
724412,0,2262057998,Sat Jun 20 21:48:48 PDT 2009,NO_QUERY,I_am_MrRebutle,Just said bye to my bro and amber .. They're g...,Spain
1298710,4,2005440663,Tue Jun 02 09:42:00 PDT 2009,NO_QUERY,kyoisorange,@laurzzzz YES. Good thinking Laura. JB CAKE B...,Dominican Republic
877146,4,1684926381,Sat May 02 22:28:50 PDT 2009,NO_QUERY,JulzM,@rperdio Can call me a bigger geek i have that...,Ethiopia
1370867,4,2051089108,Fri Jun 05 19:50:32 PDT 2009,NO_QUERY,CherylEsq,@pink82 lol,Japan
663286,0,2244062247,Fri Jun 19 14:15:11 PDT 2009,NO_QUERY,CHELSEAbbyckes,having a wicked bad carpal tunnel moment...,Russia
...,...,...,...,...,...,...,...
804202,4,1468487037,Tue Apr 07 01:58:28 PDT 2009,NO_QUERY,mdbraber,I'm now officially part of @ouroffice (or put ...,Thailand
1241765,4,1994060475,Mon Jun 01 11:21:03 PDT 2009,NO_QUERY,OhItsEmilyyy,"Yess! I agree!! Attention everyone, woahitsman...",Cuba
1595300,4,2192336508,Tue Jun 16 06:53:00 PDT 2009,NO_QUERY,joenica_danger,okay . done chatting. coz` im really tired alr...,USA
1394250,4,2053600034,Sat Jun 06 05:06:27 PDT 2009,NO_QUERY,Adoralyna,woke up with the sun after a restful sleep and...,Thailand


In [54]:
# tweets_by_location = collections.defaultdict(list)
# sentiment_sum = 0
# for row in df.iterrows():
#     sentiment = row[1][0]
#     text = row[1][5]
#     location = row[1][6]
#     sentiment_sum += sentiment
#     tweets_by_location[location].append(text)
# # print(tweets_by_location)
# print("average sentiment: ", sentiment_sum/df.size)

    

998994     @maxwellvintage I see twitter as a social part...
1580162    First 2 periods are freeee  don't have to go o...
587582        Why do they always find something else wrong? 
415282                               Watching a scary movie 
333772     @Shanicekarin i knooooooo. Omg I'm hungry. All...
                                 ...                        
1380201                         @AquaRuby sent you one back 
1597212    @timzero4 not so odd; my religion course in co...
700891     And it's time of the month. My weekend just is...
1203895                    is watching the MTV Movie Awards 
1499587    @lizzzzbeth  you will love it here. i promisee...
Name: text, Length: 6077, dtype: object


In [51]:
target_counts = collections.Counter(result[0]['target'])
location_counts = collections.Counter(result[0]['location'])
print("target counts: ", target_counts)
print("location_counts: ", location_counts)

target counts:  Counter({0: 100103, 4: 99897})
location_counts:  Counter({'Peru': 6197, 'USA': 6183, 'Canada': 6175, 'Argentina': 6167, 'Colombia': 6161, 'Germany': 6152, 'Russia': 6142, 'Indonesia': 6134, 'Japan': 6122, 'South Korea': 6116, 'Jamaica': 6094, 'Kenya': 6085, 'Italy': 6081, 'Antarctica': 6081, 'China': 6077, 'Thailand': 6071, 'Brazil': 6067, 'Cuba': 6062, 'Mexico': 6058, 'Spain': 6044, 'Nigeria': 6043, 'UK': 6023, 'Tanzania': 6021, 'Ethiopia': 6003, 'Ecuador': 6003, 'Australia': 5995, 'Dominican Republic': 5988, 'France': 5983, 'New Zealand': 5972, 'Egypt': 5958, 'India': 5933, 'South Africa': 5930, 'Chile': 5879})


In [80]:
def locationSentiment(data, location):
    """
    input: location is the input location of interest
    output: the average sentiment of the tweets for a location
    """
    # go through the dataframe filtered by location
    # get the average sentiment of the tweets in that location by getting sum and dividing by number of rows
    location_data = data[data["location"] == location]
    total_sentiment_value = sum(list(location_data["target"]))
    return total_sentiment_value/len(location_data)

def getLocationTweetsWithSentiment(data, location):
    tweets_with_sentiment = []
    location_data = data[data["location"] == location]
    for row in location_data.iterrows():
        sentiment = row[1]['target']
        text = row[1]['text']
        sentiment_for_classifier = ""
        if sentiment == 0:
            # negative
            sentiment_for_classifier = "neg"
        elif sentiment == 4:
            # positive
            sentiment_for_classifier = "pos"
        tweets_with_sentiment.append((text,sentiment_for_classifier))
    return tweets_with_sentiment

# print(locationSentiment(result[0], "China"))
tweets = getLocationTweetsWithSentiment(result[0], "China")
print(tweets)


[('@maxwellvintage I see twitter as a social party~and the unexpected could result in very positive directions for people ', 'pos'), ("First 2 periods are freeee  don't have to go on until 11! Xx", 'pos'), ('Why do they always find something else wrong? ', 'neg'), ('Watching a scary movie ', 'neg'), ("@Shanicekarin i knooooooo. Omg I'm hungry. All i got is cheerios tho ", 'neg'), ('learning Chemistry, ugh!   but in the evening going to a football match, juhuuuu! Sparta (Masters!!) vs. Bohemians (Kangaroos ) xPP', 'neg'), ('madd props to my best friend @dskotek for murdering it here at PAINT oh &amp; for him going to play his adagio for strings rmx in a bit ', 'pos'), ('@djwheil by posting this... #Squarespace i thought... ... i know i want one... would be nice huh... ', 'pos'), ('Yay  Finally got in the 1v100 beta!', 'pos'), ('After cleaning shower stall, my mind is also clean. ', 'pos'), ("Laela Marie, stfu cuz you left me alone and now i'm sitting in Brownsville with no one fun! ", '

In [53]:
# find average sentiment for each location and store in location_scores
location_scores = {}

for (location, count) in location_counts.items():
    score = locationSentiment(result[0], location)
    location_scores[location] = score

# code sourced from https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
sorted_location_scores = {k: v for k, v in sorted(location_scores.items(), key=lambda item: item[1])}
sorted_location_scores

{'Colombia': 1.9269599091056646,
 'Ecuador': 1.9430284857571214,
 'Peru': 1.9570760045183153,
 'Antarctica': 1.9687551389574083,
 'China': 1.9693927924962975,
 'Russia': 1.972647346141322,
 'Canada': 1.9731174089068826,
 'South Korea': 1.973185088293002,
 'France': 1.973591843556744,
 'Ethiopia': 1.9756788272530401,
 'UK': 1.9817366760750457,
 'Japan': 1.9836654688010453,
 'Germany': 1.9856957087126137,
 'Kenya': 1.9917830731306492,
 'Cuba': 1.9920818211811284,
 'Thailand': 1.9950584747158624,
 'Brazil': 1.9983517389154442,
 'Tanzania': 2.0023251951503074,
 'Jamaica': 2.0026255333114538,
 'Mexico': 2.002641135688346,
 'India': 2.00303387830777,
 'Nigeria': 2.0076121131888134,
 'Chile': 2.0085048477632252,
 'New Zealand': 2.010046885465506,
 'USA': 2.016496846191169,
 'Italy': 2.0220358493668806,
 'Australia': 2.0236864053377817,
 'South Africa': 2.026306913996627,
 'Argentina': 2.0314577590400518,
 'Dominican Republic': 2.0400801603206413,
 'Egypt': 2.0463242698892246,
 'Indonesia': 2.

In [35]:
# drop unnecessary columns
split_df = split_df.drop(['id', 'date', 'flag', 'user'], axis=1)

split_df

,target,text,location
328990,0,its going on a week and my dang picture is sti...,Peru
342651,0,getting ready to go to yet again,Kenya
571482,0,Leaving town. Going to my bro's house in the G...,Colombia
273030,0,"Like no one is tweeting today, this makes me sad",Peru
1074367,4,WHY is Noraniza Idris out of the spotlight?? n...,Tanzania
...,...,...,...
653806,0,is getting stressed with t-mobile,Peru
113212,0,prettyy boared.. waiting for it to stop rainin...,Indonesia
621456,0,@saxy15 awww I'm sorry I hope things get bett...,Egypt
1560312,4,"Tried her first ever glass of Almond Milk, was...",Ethiopia


In [83]:
dataset = getLocationTweetsWithSentiment(result[0], "China")
train = dataset[:int(0.8*len(dataset))] # change this so that its actually a train test split 
test = dataset[int(0.8*len(dataset)):]
cl = NaiveBayesClassifier(train) # train needs tuples of text and pos/neg
correct = 0
for text, sentiment in test:
    pred = cl.classify(text)
    correct += (pred == sentiment)
print(correct/len(test))
# cl.classify("I feel amazing!")
# blob = TextBlob("The beer is good. But the hangover is horrible.", classifier=cl)
# for s in blob.sentences:
#     print(s)
#     print(s.classify())

0.71875
